In [1]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

In [31]:
class ViewerWithCallback:

    def __init__(self, config, device, align_depth_to_color):
        self.flag_exit = False
        self.flag_reload = False
        self.align_depth_to_color = align_depth_to_color

        self.sensor = o3d.io.AzureKinectSensor(config)
        if not self.sensor.connect(device):
            raise RuntimeError('Failed to connect to sensor')

        color = o3d.camera.PinholeCameraIntrinsicParameters.Kinect2ColorCameraDefault
        depth = o3d.camera.PinholeCameraIntrinsicParameters.Kinect2DepthCameraDefault
        self.intrinsic = o3d.camera.PinholeCameraIntrinsic(color if align_depth_to_color else depth)
        self.extrinsic = o3d.camera.PinholeCameraParameters().extrinsic
            
    def escape_callback(self, vis):
        self.flag_exit = True
        return False

    def reload_callback(self, vis):
        self.flag_reload = True
        return False

    def run(self):
        self.flag_exit = False
        self.flag_reload = False

        glfw_key_escape = 256
        glfw_key_reload = 32
        vis = o3d.visualization.VisualizerWithKeyCallback()
        vis.register_key_callback(glfw_key_escape, self.escape_callback)
        vis.register_key_callback(glfw_key_reload, self.reload_callback)
        vis.create_window('viewer', 1280, 720)
        print("Sensor initialized. Press [ESC] to exit.")

        pcd = o3d.geometry.PointCloud()

        d = self.sensor.capture_frame(self.align_depth_to_color).depth
        tmp = o3d.geometry.PointCloud.create_from_depth_image(d, self.intrinsic)
        vis.add_geometry(tmp, True)
        vis.remove_geometry(tmp, False)
        
        vis_geometry_added = False
        while not self.flag_exit:
            rgbd = self.sensor.capture_frame(self.align_depth_to_color)
            if rgbd is None:
                continue

            if self.flag_reload:
                vis.remove_geometry(pcd, False)
                vis_geometry_added = False
                self.flag_reload = False
            
            if not vis_geometry_added:
                pcd = o3d.geometry.PointCloud.create_from_depth_image(rgbd.depth, self.intrinsic)
                pcd = pcd.voxel_down_sample(voxel_size=0.02)
                
                #with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
                labels = np.array(pcd.cluster_dbscan(eps=0.1, min_points=80, print_progress=False))

                max_label = labels.max()
                colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
                colors[labels < 0] = 0
                pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
                vis.add_geometry(pcd, False)
                vis_geometry_added = True

            vis.update_geometry(pcd)            
            vis.poll_events()
            vis.update_renderer()

In [33]:
config = o3d.io.AzureKinectSensorConfig()
v = ViewerWithCallback(config, 0, False)

[Open3D INFO] AzureKinectSensor::Connect
[Open3D INFO] sensor_index 0
[Open3D INFO] Serial number: 000489792412
[Open3D INFO] Firmware build: Rel
[Open3D INFO] > Color: 1.6.108
[Open3D INFO] > Depth: 1.6.79[6109.7]


In [34]:
v.run()

Sensor initialized. Press [ESC] to exit.


In [32]:
del v